In [2]:
from b2heavy.TwoPointFunctions.types2pts  import CorrelatorIO, Correlator
from b2heavy.TwoPointFunctions.fitter     import CorrFitter
from b2heavy.TwoPointFunctions.utils      import NplusN2ptModel, PeriodicExpDecay

In [3]:
import math
import autograd
import autograd.numpy    as anp
import numpy             as np
import xarray            as xr
import gvar              as gv
import pyerrors          as pr
import matplotlib.pyplot as plt

from scipy.optimize import curve_fit

In [4]:
ens = 'Coarse-1'
mes = 'Dst'
mom = '200'
binsize = 11

data_dir = '/Users/pietro/code/data_analysis/BtoD/Alex/'

In [5]:
io     = CorrelatorIO(ens,mes,mom,PathToDataDir=data_dir)
corr   = Correlator(io,jkBin=binsize)
fitter = CorrFitter(corr,smearing=['d-d','1S-1S','d-1S'])

In [28]:
NEXC = 2
TLIM = (10,17)

In [29]:
(X,meff,aeff), MEFF,AEFF, m_pr, apr = corr.EffectiveCoeff(trange=TLIM,covariance=False)

1.1413(13)


/Users/pietro/opt/anaconda3/envs/stat/lib/python3.11/site-packages/autograd/tracer.py:48: RuntimeWarning: invalid value encountered in log
  return f_raw(*args, **kwargs)


In [31]:
fitter.fit(
    Nstates           = NEXC,
    trange            = TLIM,
    verbose           = True,
    priors            = fitter.set_priors_phys(NEXC,Meff=MEFF,Aeff=AEFF),
    # scale_covariance  = True,
    # shrink_covariance = True,
    covariance        = False,
    override          = True
)

---------- 2+2 fit in (10, 17) for mes: Dst of ens: Coarse-1 for mom: 200 --------------
 De-augmented chi2/ndof [ndof] = 0.6 [25] with p-value = 0.9567649773296425
Least Square Fit:
  chi2/dof [dof] = 0.37 [48]    Q = 1    logGBF = 871.65

Parameters:
            E 0   1.1411 (13)     [ 1.1413 (13) ]  
              1    -2.56 (37)     [  -1.81 (58) ]  *
              2    -1.11 (51)     [  -1.09 (80) ]  
              3    -2.07 (68)     [  -2.4 (2.5) ]  
     Z_1S_Bot 0    0.165 (15)     [   0.13 (45) ]  
              1    -0.14 (23)     [  -1.2 (1.2) ]  
              2     0.71 (90)     [   0.5 (1.5) ]  
              3     0.2 (1.3)     [   0.5 (1.5) ]  
     Z_1S_Par 0    0.240 (17)     [   0.24 (39) ]  
              1    -0.50 (35)     [  -1.2 (1.2) ]  
              2     1.21 (94)     [   0.5 (1.5) ]  
              3     0.4 (1.0)     [   0.5 (1.5) ]  
   Z_d-1S_Bot 0     0.36 (27)     [   0.5 (1.7) ]  
              1     0.28 (13)     [   0.5 (1.7) ]  
   Z_d-1S_Par 0   

# New function

In [15]:
def _model_single(t,p,Nstates,Nt):
    rlen = 2*Nstates
    ans  = anp.exp(p[rlen  ])**2   * ( anp.exp(-p[0] * t) + anp.exp(-p[0] * (Nt-t)) )

    erg = [p[0]]
    for n in range(1,2*Nstates):
        En = anp.exp(p[n]) + erg[n - (1 if n==1 else 2)]
        ans += anp.exp(p[rlen+n])**2 * ( anp.exp(-En*t) + anp.exp(-En*(Nt-t)) ) * (-1)**(n*(t+1))
        erg.append(En)

    return ans

def model_single(Nstates,Nt):
    return lambda x,p : _model_single(x,p,Nstates,Nt)

In [16]:
def _model_mixed(t,p,Nstates,Nt):
    rlen = 2*Nstates

    e0,e1 = p[0],p[0]+anp.exp(p[1])
    ans  = anp.exp(p[rlen  ]) * anp.exp(p[2*rlen  ])   * ( anp.exp(-e0 * t) + anp.exp(-e0 * (Nt-t)) )
    ans += anp.exp(p[rlen+1]) * anp.exp(p[2*rlen+1])   * ( anp.exp(-e1 * t) + anp.exp(-e1 * (Nt-t)) ) * (-1)**(t+1)

    erg = [e0,e1]
    for n in range(2,2*Nstates):
        En = anp.exp(p[n]) + erg[n-2]
        ans += anp.exp(p[3*rlen+(n-2)])**2 * ( anp.exp(-En*t) + anp.exp(-En*(Nt-t)) ) * (-1)**(n*(t+1))

        erg.append(En)
    
    return ans

def model_mixed(Nstates,Nt):
    return lambda x,p : _model_mixed(x,p,Nstates,Nt)

In [50]:
Nstates = NEXC
Nt      = corr.Nt
Npol    = len(corr.data.polarization)

func = {}
for i,(sm,pol) in enumerate(fitter.keys):
    sm1,sm2 = sm.split('-')
    mix = sm1!=sm2

    def _model(t,p):
        rlen = 2*Nstates
        pars = anp.array(p[:rlen])
        if mix:
            i1 = rlen + (i%Npol)*rlen
            pars = anp.append(pars,p[i1:(i1+rlen)])

            i2 = i1 + Npol*rlen
            pars = anp.append(pars,p[i2:(i2+rlen)])

            im = rlen + 2*Npol*rlen + (i%Npol)*(rlen-2)
            pars = anp.append(pars,p[im:im+(rlen-2)])

            return model_mixed(Nstates,Nt)(t,anp.array(pars))
            
        else:
            I = rlen + (i if i<Npol else i-2)*rlen            
            pars = anp.append(pars,p[I:(I+rlen)])
            
            return model_single(Nstates,Nt)(t,anp.array(pars))

    func[sm,pol] = _model

In [51]:
def dict_to_array(pdict):
    pars = [pdict['E']]

    smr = set()
    pol = set()
    for k in pdict:
        if k.startswith('Z'):
            _,sm,pl = k.split('_')
            smr.add(sm)
            pol.add(pl)

    for s in sorted(smr):
        for p in sorted(pol):
            pars.append(pdict[f'Z_{s}_{p}'])

    return np.concatenate(pars)

pars = dict_to_array(fit.p)

In [56]:
xdata,ydata = corr.format(trange=TLIM,smearing=['d-d','d-1S','1S-1S'])

In [78]:
k = ('1S-1S','Par')


np.concatenate([func[k](xdata[k],gv.mean(pars)) for k in fitter.keys])

array([2.73381706e-08, 5.28787046e-08, 2.87201771e-09, 2.46090569e-09,
       2.52281889e-10, 1.30345178e-10, 2.02557759e-11, 7.75621136e-12,
       1.55039171e-12, 5.02064484e-13, 2.73381706e-08, 5.28787046e-08,
       2.87201771e-09, 2.46090569e-09, 2.52281889e-10, 1.30345178e-10,
       2.02557759e-11, 7.75621136e-12, 1.55039171e-12, 5.02064484e-13,
       2.73381706e-08, 5.28787046e-08, 2.87201771e-09, 2.46090569e-09,
       2.52281889e-10, 1.30345178e-10, 2.02557759e-11, 7.75621136e-12,
       1.55039171e-12, 5.02064484e-13, 2.73381706e-08, 5.28787046e-08,
       2.87201771e-09, 2.46090569e-09, 2.52281889e-10, 1.30345178e-10,
       2.02557759e-11, 7.75621136e-12, 1.55039171e-12, 5.02064484e-13,
       2.73381706e-08, 5.28787046e-08, 2.87201771e-09, 2.46090569e-09,
       2.52281889e-10, 1.30345178e-10, 2.02557759e-11, 7.75621136e-12,
       1.55039171e-12, 5.02064484e-13, 2.73381706e-08, 5.28787046e-08,
       2.87201771e-09, 2.46090569e-09, 2.52281889e-10, 1.30345178e-10,
      

In [81]:
pars

array([1.3138(26), -2.28(43), -0.93(34), -1.43(67), 0.136(20), -0.29(33),
       1.8(1.1), 0.9(1.1), 0.231(15), -0.45(25), 0.2(1.5), 0.2(1.5),
       -1.811(22), -2.41(38), 0.43(26), 0.17(16), -1.737(20), -6.0(2.0),
       0.38(24), 0.28(24), 0.91(52), 0.42(34), 0.68(42), 0.67(56)],
      dtype=object)

# New new function

In [76]:
Nstates = NEXC
trange  = TLIM
Nt      = corr.Nt

In [77]:
def _model(t,E0,Z0a,Z0b, E1,Z1a,Z1b, *high):
    C_t  = anp.exp(Z0a) * anp.exp(Z0b) * (anp.exp(-E0*t) + anp.exp(-E0*(Nt-t)))
    C_t += anp.exp(Z1a) * anp.exp(Z1b) * (anp.exp(-E1*t) + anp.exp(-E1*(Nt-t))) * (-1)**(t+1)
    for n in range(2*Nstates-2):
        C_t += high[2*n+1]**2 * ( anp.exp(-high[2*n]*t) + anp.exp(-high[2*n]*(Nt-t))) * (-1)**(n*(t+1))
    return C_t

In [78]:
def Smr(sm):
    s1,s2 = sm.split('-')
    return s1 if s1==s2 else sm 

def Energies(Evec):
    erg = [Evec[0]]
    for n in range(1,len(Evec)):
        erg.append(anp.exp(Evec[n]) + erg[n-(1 if n==1 else 2)])
    return erg

In [79]:
pars_keys = ['E',*[f'Z_{Smr(sm)}_{pol}' for sm,pol in fitter.keys]]

In [86]:
keys = [(k,exc) for exc in range(2*Nstates) for k in pars_keys if not ('-' in k and exc<2)]

In [87]:
keys

[('E', 0),
 ('Z_1S_Bot', 0),
 ('Z_1S_Par', 0),
 ('Z_d_Bot', 0),
 ('Z_d_Par', 0),
 ('E', 1),
 ('Z_1S_Bot', 1),
 ('Z_1S_Par', 1),
 ('Z_d_Bot', 1),
 ('Z_d_Par', 1),
 ('E', 2),
 ('Z_1S_Bot', 2),
 ('Z_1S_Par', 2),
 ('Z_d-1S_Bot', 2),
 ('Z_d-1S_Par', 2),
 ('Z_d_Bot', 2),
 ('Z_d_Par', 2),
 ('E', 3),
 ('Z_1S_Bot', 3),
 ('Z_1S_Par', 3),
 ('Z_d-1S_Bot', 3),
 ('Z_d-1S_Par', 3),
 ('Z_d_Bot', 3),
 ('Z_d_Par', 3),
 ('E', 4),
 ('Z_1S_Bot', 4),
 ('Z_1S_Par', 4),
 ('Z_d-1S_Bot', 4),
 ('Z_d-1S_Par', 4),
 ('Z_d_Bot', 4),
 ('Z_d_Par', 4),
 ('E', 5),
 ('Z_1S_Bot', 5),
 ('Z_1S_Par', 5),
 ('Z_d-1S_Bot', 5),
 ('Z_d-1S_Par', 5),
 ('Z_d_Bot', 5),
 ('Z_d_Par', 5)]

In [81]:
def pars_from_dict(pdict):
    erg = Energies(pdict['E'])
    return anp.array([erg[n] if k=='E' else pdict[k][n-(2 if '-' in k else 0)] for k,n in keys])

In [103]:
xdata,ydata = corr.format(trange=TLIM,smearing=['1S-1S','d-d','d-1S'])

def flat_timeslices(xdict):
    aux = [xdata[k]*(10**i) for i,k in enumerate(fitter.keys)]
    return np.concatenate(aux)

# print(int(math.log(x,10))-1)

In [ ]:
def new_model(flatT,params):


In [85]:
def flatten_model(T, params):
    ymodel = []
    for ik,(sm,pol) in enumerate(fitter.keys):
        sm1,sm2 = sm.split('-')
        
        iipars = [ # Collect parameters for fund_phys and fund_osc
            keys.index(('E',0)),keys.index((f'Z_{sm1}_{pol}',0)),keys.index((f'Z_{sm2}_{pol}',0)),
            keys.index(('E',1)),keys.index((f'Z_{sm1}_{pol}',1)),keys.index((f'Z_{sm2}_{pol}',1))
        ]
        for exc in range(2,2*Nstates): # Collect parameters for excited states
            iipars.extend([keys.index(('E',exc)),keys.index((f'Z_{sm if sm1!=sm2 else sm1}_{pol}',exc))])

        pars = anp.array(params)[iipars]
        park = [keys[i] for i in iipars]

        ymodel.extend(_model(T[sm,pol],*pars))

    return anp.array(ymodel)

def wrapper_flatten_model(T,*params):
    return wrapper_flatten_model(T,params)

In [83]:
xdata,ydict = corr.format(trange=TLIM)
p0 = pars_from_dict(fit.pmean)
wrapper_flatten_model(xdata,*p0);

In [84]:
dfdp = []
for i in range(1,len(p0)+1):
    dfdp.append(autograd.grad(wrapper_flatten_model,i))

for f in dfdp:
    f(xdata,*p0)

TypeError: Grad only applies to real scalar-output functions. Try jacobian, elementwise_grad or holomorphic_grad.

# New New implementation

In [33]:
Nstates = NEXC
trange  = TLIM
Nt      = corr.Nt

def _model(t,E0,Z0, E1,Z1, *high):
    C_t  = Z0 * (anp.exp(-E0*t) + anp.exp(-E0*(Nt-t)))
    C_t += Z1 * (anp.exp(-E1*t) + anp.exp(-E1*(Nt-t))) * (-1)**(t+1)
    for n in range(2*Nstates-2):
        C_t += high[2*n+1] * ( anp.exp(-high[2*n]*t) + anp.exp(-high[2*n]*(Nt-t))) * (-1)**(n*(t+1))
    return C_t

In [34]:
def Smr(sm):
    s1,s2 = sm.split('-')
    return s1 if s1==s2 else sm 
    
pars_keys = ['E',*[f'Z_{Smr(sm)}_{pol}' for sm,pol in fitter.keys]]
fpars = [(k,exc) for exc in range(2*Nstates) for k in pars_keys if not ('-' in k and exc<2)]

In [35]:
fit = fitter.fits[Nstates,trange]
fitpars = [fit.p[k][n-2 if ('-' in k and n>1) else n] for k,n in fpars] # questi sono tutti i parameteri srotolati

In [47]:
idx = fpars.index(('Z_1S_Bot',2))
fitpars[idx]

0.71(90)

In [48]:
fit.p['Z_1S_Bot']

array([0.165(15), -0.14(23), 0.71(90), 0.2(1.3)], dtype=object)

In [49]:
# dato una chiave (sm,pol) devo chiamare _model() con i parametri giusti
def pars_from_key(sm,pol):
    sm1,sm2 = sm.split('-')
    
    E0  = fitpars[fpars.index(('E',0))]
    z0a = fitpars[fpars.index((f'Z_{sm1}_{pol}',0))]
    z0b = fitpars[fpars.index((f'Z_{sm2}_{pol}',0))]
    Z0 = anp.exp( z0a ) * anp.exp( z0b )
   
    E1 = E0 + anp.exp(fitpars[fpars.index(('E',1))])
    z1a = fitpars[fpars.index((f'Z_{sm1}_{pol}',1))]
    z1b = fitpars[fpars.index((f'Z_{sm2}_{pol}',1))]
    Z1 = anp.exp( z1a ) * anp.exp( z1b )
    
    erg = [E0,E1]

    p = [E0,Z0,E1,Z1]
    for n in range(2,2*Nstates):
        erg.append(
            erg[n-2] + anp.exp(fitpars[fpars.index(('E',n))])
        )
        Zn = anp.exp( fitpars[fpars.index((f'Z_{sm if sm1!=sm2 else sm1}_{pol}',n))] )**2
        p.extend([erg[-1],Zn])

    return p



[1.1411(13),
 1.617(55),
 1.218(28),
 0.36(25),
 1.47(17),
 11(21),
 1.344(80),
 2.2(4.5)]

In [53]:
_model(fit.x[0],*pars_from_key('1S-1S','Par'))

array([1.74(65)e-05, 8.2(2.2)e-06, 1.69(41)e-06, 7.4(1.3)e-07,
       1.70(27)e-07, 7.06(84)e-08, 1.74(18)e-08, 6.87(53)e-09],
      dtype=object)

In [54]:
fit.fcn(fit.x,fit.p)

array([1.165(18)e-05, 6.127(78)e-06, 1.238(23)e-06, 6.056(92)e-07,
       1.309(35)e-07, 6.02(14)e-08, 1.378(44)e-08, 6.01(17)e-09,
       1.641(20)e-05, 6.469(77)e-06, 1.680(27)e-06, 6.43(10)e-07,
       1.728(40)e-07, 6.45(15)e-08, 1.780(50)e-08, 6.49(19)e-09,
       1.459(17)e-06, 6.864(75)e-07, 1.538(17)e-07, 6.732(73)e-08,
       1.615(24)e-08, 6.67(10)e-09, 1.687(31)e-09, 6.66(13)e-10,
       1.992(21)e-06, 7.262(83)e-07, 2.052(21)e-07, 7.192(81)e-08,
       2.113(28)e-08, 7.20(10)e-09, 2.173(33)e-09, 7.26(12)e-10,
       1.830(16)e-07, 7.925(78)e-08, 1.886(22)e-08, 7.671(95)e-09,
       1.954(32)e-09, 7.56(14)e-10, 2.024(41)e-10, 7.53(17)e-11,
       2.302(19)e-07, 8.513(86)e-08, 2.382(26)e-08, 8.41(10)e-09,
       2.459(38)e-09, 8.41(15)e-10, 2.531(44)e-10, 8.47(17)e-11],
      dtype=object)

# New implementation

In [84]:
def _model_other(t,E0,Z0a,Z0b, E1,Z1a,Z1b, *high):
    C_t  = anp.exp(Z0a) * anp.exp(Z0b) * (anp.exp(-E0*t) + anp.exp(-E0*(Nt-t)))
    C_t += anp.exp(Z1a) * anp.exp(Z1b) * (anp.exp(-E1*t) + anp.exp(-E1*(Nt-t))) * (-1)**(t+1)
    for n in range(2*Nstates-2):
        C_t += high[2*n+1]**2 * ( anp.exp(-high[2*n]*t) + anp.exp(-high[2*n]*(Nt-t))) * (-1)**(n*(t+1))
    return C_t

In [9]:
def Smr(sm):
    s1,s2 = sm.split('-')
    return s1 if s1==s2 else sm 
    
pars_keys = ['E',*[f'Z_{Smr(sm)}_{pol}' for sm,pol in fitter.keys]]
flat_pars = [(k,exc) for exc in range(2*Nstates) for k in pars_keys if not ('-' in k and exc<2)]

In [55]:
def flat_model(tau,p):
    ik = int(math.log10(tau))-1 # fitter.keys[ik] gives (smr,pol)

    sm,pol = fitter.keys[ik]
    s1,s2 = sm.split('-')
    mix = s1!=s2

    # Fetch and build energies from p
    En = [p[flat_pars.index(('E',0))]]
    for n in range(1,2*Nstates):
        En.append(
            En[0 if n==1 else n-2] + anp.exp( p[flat_pars.index(('E',n))] )
        )

    # Fetch and build overlap coeffs from p
    Z0 = anp.exp(p[flat_pars.index( (f'Z_{s1}_{pol}',0) )]) * anp.exp(p[flat_pars.index( (f'Z_{s2}_{pol}',0) )])
    Z1 = anp.exp(p[flat_pars.index( (f'Z_{s1}_{pol}',1) )]) * anp.exp(p[flat_pars.index( (f'Z_{s2}_{pol}',1) )])

    high = []
    for n in range(2,2*Nstates):
        Z = anp.exp(p[flat_pars.index( (f'Z_{sm if mix else s1}_{pol}',n) )])
        high.extend([En[n],Z**2])

    # Rebuild original time
    t = tau/10**ik

    # return _model(t,En[0],Z0,En[1],Z1,*high) 
    return [En[0],Z0,En[1],Z1,*high]   

In [79]:
fit = fitter.fits[Nstates,trange]
pp = [fit.p[k][n-2 if ('-' in k and n>1) else n] for k,n in flat_pars]

E0,Z0,E1,Z1,E2,Z2,E3,Z3,E4,Z4,E5,Z5 = flat_model(10,pp)

1S-1S Bot


In [81]:
def flat_timeslices(xdict):
    aux = [xdata[k]*(10**i) for i,k in enumerate(fitter.keys)]
    return np.concatenate(aux)

In [82]:
fit.fcn(fit.x,fit.p)

array([1.166(18)e-05, 6.120(78)e-06, 1.231(24)e-06, 6.082(92)e-07,
       1.298(36)e-07, 6.07(14)e-08, 1.365(45)e-08, 6.09(17)e-09,
       1.430(53)e-09, 6.12(20)e-10, 1.640(20)e-05, 6.468(76)e-06,
       1.682(27)e-06, 6.473(95)e-07, 1.736(38)e-07, 6.52(14)e-08,
       1.793(47)e-08, 6.59(17)e-09, 1.851(53)e-09, 6.68(19)e-10,
       1.457(17)e-06, 6.865(77)e-07, 1.546(25)e-07, 6.746(95)e-08,
       1.620(32)e-08, 6.73(13)e-09, 1.687(37)e-09, 6.77(15)e-10,
       1.752(42)e-10, 6.83(16)e-11, 1.991(22)e-06, 7.263(85)e-07,
       2.059(30)e-07, 7.20(11)e-08, 2.121(38)e-08, 7.25(14)e-09,
       2.180(42)e-09, 7.36(16)e-10, 2.239(44)e-10, 7.48(16)e-11,
       1.830(16)e-07, 7.920(80)e-08, 1.884(27)e-08, 7.69(11)e-09,
       1.947(34)e-09, 7.65(15)e-10, 2.014(48)e-10, 7.70(18)e-11,
       2.082(62)e-11, 7.78(21)e-12, 2.303(19)e-07, 8.512(87)e-08,
       2.377(31)e-08, 8.41(12)e-09, 2.449(40)e-09, 8.46(16)e-10,
       2.520(51)e-10, 8.57(18)e-11, 2.591(60)e-11, 8.71(20)e-12],
      dtype=obj